**BITPANDA IMPORT CSVDATA for DCA cost INFORMATION**

In [2]:
from modules import general_utils
from modules.general_utils import glob, os, np, pd, dt, go, sp, yf
from modules import collect_data_utils
from modules.collect_data_utils import get_current_user,collect_data_from_csv,collect_bitpanda_data,sort_data_by_asset,collect_data_from_list_csv,collect_numb_sample,collect_file

path = f"C:\\Users\\{get_current_user()}\\Downloads\\Telegram Desktop\\bitpanda-trades-2025-05-02-09-36.csv"


In [ ]:
multiple_files = 0

# Assuming data_collected is a dictionary returned by collect_bitpanda_data
data_collected = collect_bitpanda_data(path)
data_collected = sort_data_by_asset(data_collected)
# Convert the relevant columns to numeric types
df = pd.DataFrame(data_collected)
df['amount_asset_collect'] = pd.to_numeric(df['amount_asset_collect'], errors='coerce')
df['amount_fiat_collect'] = pd.to_numeric(df['amount_fiat_collect'], errors='coerce')
df['asset_market_price_collect'] = pd.to_numeric(df['asset_market_price_collect'], errors='coerce')

# Filter out 'stake' and 'transfer' transactions
df = df[~df['trans_type_collect'].isin(['stake', 'transfer'])]

# Adjust the amounts for 'sell' transactions
df.loc[df['trans_type_collect'] == 'sell', 'amount_asset_collect'] *= -1

# Store the deposit entry for BTC
btc_deposit_amount = df.loc[(df['asset_collect'] == 'BTC') & (df['trans_type_collect'] == 'deposit'), 'amount_asset_collect'].sum()
btc_deposit_median_price = 34279.00569

# Remove 'deposit' entries for BTC from the main DataFrame
df = df[~((df['asset_collect'] == 'BTC') & (df['trans_type_collect'] == 'deposit'))]

# Group by 'asset_collect' and calculate the sum of 'amount_asset_collect' and 'amount_fiat_collect'
grouped = df.groupby('asset_collect').agg({
    'amount_asset_collect': 'sum',
    'amount_fiat_collect': 'sum'
}).reset_index()

# Calculate the median price for each asset
grouped['median_price'] = grouped['amount_fiat_collect'] / grouped['amount_asset_collect']

# Adjust the BTC entry to include the deposit data
if 'BTC' in grouped['asset_collect'].values:
    btc_row = grouped[grouped['asset_collect'] == 'BTC']
    btc_total_amount = btc_row['amount_asset_collect'].values[0] + btc_deposit_amount
    btc_total_fiat = btc_row['amount_fiat_collect'].values[0] + (btc_deposit_amount * btc_deposit_median_price)
    btc_final_median_price = btc_total_fiat / btc_total_amount

    # Update the BTC row in the grouped DataFrame
    grouped.loc[grouped['asset_collect'] == 'BTC', 'amount_asset_collect'] = btc_total_amount
    grouped.loc[grouped['asset_collect'] == 'BTC', 'amount_fiat_collect'] = btc_total_fiat
    grouped.loc[grouped['asset_collect'] == 'BTC', 'median_price'] = btc_final_median_price
else:
    # If BTC is not in the grouped DataFrame, add it
    grouped = pd.concat([
        grouped,
        pd.DataFrame({
            'asset_collect': ['BTC'],
            'amount_asset_collect': [btc_deposit_amount],
            'amount_fiat_collect': [btc_deposit_amount * btc_deposit_median_price],
            'median_price': [btc_deposit_median_price]
        })
    ], ignore_index=True)

# Print the resulting DataFrame with the calculated sums and median prices
print(grouped)



  asset_collect  amount_asset_collect  amount_fiat_collect  median_price
0           BNB              0.000023                24.44  1.076652e+06
1           BTC              0.008435               839.59  9.953834e+04
2          DOGE            953.392548                 0.00  0.000000e+00
3           ETH              0.000008                86.50  1.034689e+07
4           EUR              0.000000              1429.01           inf
5           SOL              7.499973              1295.71  1.727620e+02
6         TRUMP              1.000000                68.62  6.862000e+01
7           XRP              0.007261                34.97  4.815836e+03


c:\Users\Davidde\Documents\MyGITprj\modules\collect_data_utils.py:203: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  collected_data[key].append(row[column_index])
